In [1]:
import pickle
import numpy as np
with open("train_qa.txt","rb") as f:
    train_data = pickle.load(f)
    

In [2]:
with open("test_qa.txt","rb") as f:
    test_data = pickle.load(f)
    

In [3]:
all_data = test_data+train_data


In [4]:
vocab = set()
for story,ques,ans in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(ques))

In [5]:
vocab.add("yes")
vocab.add("no")

In [6]:
len(vocab)

37

In [7]:
vocab_len = len(vocab)+1

In [8]:
max_story_len = max([len(data[0])for data in all_data])
max_story_len
   

156

In [9]:
max_ques_len = max([len(data[1])for data in all_data])
max_ques_len
    

6

In [10]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install tensorflow

In [14]:
pip install keras.preprocessing.text

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement keras.preprocessing.text (from versions: none)
ERROR: No matching distribution found for keras.preprocessing.text


In [17]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer




ModuleNotFoundError: No module named 'keras.preprocessing.text'

In [16]:
tokenizer = Tokenizer(filters = [])


NameError: name 'Tokenizer' is not defined

In [ ]:
tokenizer.fit_on_texts(vocab)

In [ ]:
train_story_text,train_question_text,train_answers = [],[],[]
for story,question,answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [ ]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)
len(train_story_seq)

In [ ]:
tokenizer.word_index

In [ ]:
def vectorize_stories(data,word_index = tokenizer.word_index,
                      max_story_len = max_story_len,max_ques_len = max_ques_len):
    X = []
    Xq = []
    Y = []
    for story,query,answer in data:
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y = np.zeros(len(word_index)+1)
        y[word_index[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return(pad_sequences(X,maxlen = max_story_len),
          pad_sequences(Xq,maxlen = max_ques_len),
          np.array(Y))

    

In [ ]:
inputs_train,query_train,answers_train = vectorize_stories(train_data)

In [ ]:
inputs_test,query_test,answers_test = vectorize_stories(test_data)

In [ ]:
answers_test

In [ ]:
tokenizer.word_index['yes']

In [ ]:
tokenizer.word_index['no']

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Embedding
from keras.layers import *

In [ ]:
input_sequence = Input((max_story_len,))
question = Input((max_ques_len,))


In [ ]:
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_len,output_dim = 64))
input_encoder_m.add(Dropout(0.3))

In [ ]:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_len,output_dim = max_ques_len))
input_encoder_c.add(Dropout(0.3))

In [ ]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = vocab_len,output_dim = 64,input_length = max_ques_len))
question_encoder.add(Dropout(0.3))

In [ ]:
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [ ]:
match = dot([input_encoded_m,question_encoded],axes = (2,2))
match = Activation('softmax')(match)

In [ ]:
response = add([match,input_encoded_c])
response = Permute((2,1))(response)


In [ ]:
answer = concatenate([response,question_encoded])


In [ ]:
answer

In [ ]:
answer = LSTM(32)(answer)


In [ ]:
answer

In [ ]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)

In [ ]:
answer = Activation("softmax")(answer)

In [ ]:
model = Model([input_sequence,question],answer)
model.compile(optimizer = 'rmsprop',loss = 'categorical_crossentropy',metrics = ["accuracy"])


In [ ]:
model.summary()

In [ ]:
history = model.fit([inputs_train,query_train],answers_train,
                   batch_size = 32,epochs = 20,
                   validation_data = ([inputs_test,query_test],answers_test))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.title("Model accuracy")
plt.xlabel("epochs")
plt.ylabel("Accuracy")

In [ ]:
from keras.models import save_model
model.save("Chatbot.h5")

In [ ]:
model.load_weights("chatbot.h5")
results = model.predict(([inputs_test,query_test]))

In [ ]:
val_max = np.argmax(results[23])

In [ ]:
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print("Predicted answer:",k)
print("Probability:",results[23][val_max])

In [ ]:
test_data[0][0]

In [ ]:
story = ' '.join(test_data[14][0])
story

In [ ]:
question = ' '.join(test_data[14][1])
question

In [ ]:
test_data[14][2]

In [ ]:
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print("Predicted answer:",k)
print("Probability:",results[23][val_max])

In [ ]:
story = "Mary dropped the football . Sandra discarded apple in kitchen . Daniel went to the office . "
question = "Is Daniel in the office ?"
mydata = [(story.split(),question.split(),"yes")]


In [ ]:
my_story,my_question,my_answer = vectorize_stories(mydata)

In [ ]:
pred_results = model.predict(([my_story,my_question]))
val_max = np.argmax(pred_results[0])
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key
print("Predicted answer:",k)
print("Probability:",pred_results[0][val_max])